<a href="https://colab.research.google.com/github/Gmai/AssistenteDeMercado/blob/main/AssistenteDeMercado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este codigo e um MVP de um aplicativo em que voce informa o cardapio da sua semana e ele gera a lista de compras
Em seguida e possivel tirar fotos da sua dispensa, geladeira, etc...
E ele extrai os items das imagens e subtrai da lista de compra, gerando uma lista final de compras

Importacoes

In [14]:
"""
At the command line, only need to run once to install the package via pip:

$ pip install google-generativeai
"""
from ipywidgets import FileUpload
from pathlib import Path
import hashlib
import google.generativeai as genai
from google.colab import userdata
import tempfile

Carregando Key da api do Gemini

In [2]:
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

Setup dos modelos

In [3]:

# Set up the model
generation_config_chat = {
  "temperature": 0.5,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

generation_config_structured = {
  "temperature": 0.2,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model_chat = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config_chat,
                              safety_settings=safety_settings)
model_structured = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config_structured,
                              safety_settings=safety_settings)

Input das refeicoes que seram feitas durante a semana, em caso de nao querer registar uma refeicao ou nao for comer em casa, basta dar enter e passar para o proximo

In [27]:
quantidadeDePessoas = input("Quantas pessoas vao comer?")
print("Segunda-feira")
segundaFeira = [input("Cafe da manha?"),input("Almoco?"),input("Janta?")]
print("Terca-feira")
tercaFeira = [input("Cafe da manha?"),input("Almoco?"),input("Janta?")]
print("Quarta-feira")
quartaFeira = [input("Cafe da manha?"),input("Almoco?"),input("Janta?")]
print("Quinta-feira")
quintaFeira = [input("Cafe da manha?"),input("Almoco?"),input("Janta?")]
print("Sexta-feira")
sextaFeira = [input("Cafe da manha?"),input("Almoco?"),input("Janta?")]
print("Sabado")
sabado = [input("Cafe da manha?"),input("Almoco?"),input("Janta?")]
print("Domingo")
domingo = [input("Cafe da manha?"),input("Almoco?"),input("Janta?")]

Quantas pessoas vao comer?3
Segunda-feira
Cafe da manha?pao de queijo
Almoco?strogonoff
Janta?parmegiana
Terca-feira
Cafe da manha?pao com manteiga
Almoco?
Janta?
Quarta-feira
Cafe da manha?
Almoco?
Janta?macarrao alho e oleo
Quinta-feira
Cafe da manha?
Almoco?
Janta?
Sexta-feira
Cafe da manha?
Almoco?
Janta?
Sabado
Cafe da manha?
Almoco?
Janta?
Domingo
Cafe da manha?
Almoco?
Janta?


Metodo auxliar para gerar query para o Gemini

In [28]:
def buildCardapioString(__dia,  __refeicoes):
  cardapioString=__dia+":"
  if __refeicoes[0] != "":
    cardapioString += "cafe da manha="+__refeicoes[0]+";"
  if __refeicoes[1] != "":
    cardapioString += "almoco="+__refeicoes[1]+";"
  if __refeicoes[2] != "":
    cardapioString += "janta="+__refeicoes[2]+";"
  if cardapioString != __dia+":":
    return cardapioString
  else:
    return ""

Gerando a query para o Gemini

In [31]:
cardapio=""
cardapio += buildCardapioString("Segunda-feira", segundaFeira)
cardapio += buildCardapioString("Terca-feira", tercaFeira)
cardapio += buildCardapioString("Quarta-feira", quartaFeira)
cardapio += buildCardapioString("Quinta-feira", quintaFeira)
cardapio += buildCardapioString("Sexta-feira", sextaFeira)
cardapio += buildCardapioString("Sabado", sabado)
cardapio += buildCardapioString("Domingo", domingo)
cardapio

'Segunda-feira:cafe da manha=pao de queijo;almoco=strogonoff;janta=parmegiana;Terca-feira:cafe da manha=pao com manteiga;Quarta-feira:janta=macarrao alho e oleo;'

Executando query de calculo da lista de compras

In [32]:
convo = model_chat.start_chat(history=[])

convo.send_message( "eu quero que voce me de em pt-BR a lista de compras necessarias para conseguir fazer o seguinte cardapio, para"+
        quantidadeDePessoas+
        " pessoas que comem normal, estime as quantidades"+
                    cardapio)
print(convo.last.text)

## Lista de Compras para 3 pessoas:

**Segunda-feira:**

* **Café da manhã:**
    * 1 pacote de pão de queijo (500g)
* **Almoço (Strogonoff):**
    * 500g de carne (filé mignon, alcatra ou frango)
    * 1 cebola grande
    * 2 dentes de alho
    * 1 caixa de creme de leite
    * 1 sachê de molho de tomate
    * 1 vidro de champignon
    * Azeite
    * Sal e pimenta do reino a gosto
    * Arroz (quantidade de acordo com a preferência)
    * Batata palha (opcional)
* **Janta (Parmegiana):**
    * 4 bifes grandes (contrafilé ou filé mignon)
    * 2 ovos
    * Farinha de rosca
    * Queijo mussarela fatiado
    * Molho de tomate
    * Queijo parmesão ralado
    * Azeite
    * Sal e pimenta do reino a gosto
    * Arroz (quantidade de acordo com a preferência)

**Terça-feira:**

* **Café da manhã:**
    * 1 pacote de pão francês (5 unidades)
    * Manteiga

**Quarta-feira:**

* **Janta (Macarrão alho e óleo):**
    * 500g de macarrão (espaguete ou penne)
    * 4 dentes de alho
    * Azeite
 

Eu poderia ter tentado gerar a lista somente com o chat, mas eu quis testar o uso do "Structured prompt", entao aqui eu pego o output descritivo do chat e extraio somente a lista de items

In [33]:
prompt_parts = [
  "retorne somente a lista enumerada",
  "input: **Segunda-feira:**\n\n* **Pão francês:** 18 unidades (6 por pessoa)\n* **Queijo minas frescal:** 500g\n* **Berinjela:** 3 unidades médias\n* **Molho de tomate:** 1 garrafa\n* **Queijo muçarela:** 200g\n* **Presunto:** 200g\n* **Carne moída:** 500g\n* **Macarrão para yakissoba:** 500g\n* **Legumes para yakissoba (cenoura, brócolis, couve-flor, etc.):** a gosto\n\n**Terça-feira:**\n\n* **Pão francês:** 18 unidades (6 por pessoa)\n* **Requeijão:** 1 pote grande\n* **Ingredientes para pizza (massa pronta, molho de tomate, queijo muçarela, etc.):** a gosto\n\n**Itens Básicos (se necessário):**\n\n* **Óleo:** 1 litro\n* **Azeite:** 250ml\n* **Alho:** 1 cabeça\n* **Cebola:** 1kg\n* **Sal:** a gosto\n* **Pimenta-do-reino:** a gosto\n* **Temperos diversos:** a gosto (orégano, manjericão, etc.)\n\n**Observações:**\n\n* A quantidade de pão pode variar conforme o apetite de cada pessoa.\n* Se você preferir fazer a massa da pizza em casa, inclua os ingredientes necessários na lista.\n* Adapte a lista de acordo com suas preferências e necessidades. \n\n**Espero que essa lista de compras te ajude a preparar o cardápio da semana! Bom apetite!**",
  "output: 1 - Pão francês: 18 unidades (6 por pessoa)\n2 - Queijo minas frescal: 500g\n3 - Berinjela: 3 unidades médias\n4 - Molho de tomate: 1 garrafa\n5 - Queijo muçarela: 200g\n6 - Presunto: 200g\n7 - Carne moída: 500g\n8 - Macarrão para yakissoba: 500g\n9 - Legumes para yakissoba (cenoura, brócolis, couve-flor, etc.): a gosto\n10 - Pão francês: 18 unidades (6 por pessoa)\n11 - Requeijão: 1 pote grande\n12 - Ingredientes para pizza (massa pronta, molho de tomate, queijo muçarela, etc.): a gosto\n13 - Óleo: 1 litro\n14 - Azeite: 250ml\n15 - Alho: 1 cabeça\n16 - Cebola: 1kg\n17 - Sal: a gosto\n18 - Pimenta-do-reino: a gosto\n19 - Temperos diversos: a gosto (orégano, manjericão, etc.)",
  "input: ## Lista de compras para o cardápio (4 pessoas):\n\n**Segunda-feira:**\n\n*   **Café da manhã:**\n    *   Cereal (escolha o tipo e a quantidade de acordo com a preferência, ex: 1 caixa de cereal de milho)\n    *   Leite (1 litro) \n*   **Jantar:**\n    *   Macarrão (500g)\n    *   Alho (1 cabeça)\n    *   Óleo (a gosto)\n    *   Queijo ralado (a gosto)\n\n**Terça-feira:**\n\n*   **Café da manhã:**\n    *   Polvilho doce (500g)\n    *   Queijo minas padrão ou meia cura ralado (200g)\n    *   Ovos (2 unidades)\n    *   Óleo (para untar)\n    *   Leite (1/2 xícara)\n*   **Almoço:**\n    *   Filés de frango (4 unidades)\n    *   Ovos (2 unidades)\n    *   Farinha de trigo (1 xícara)\n    *   Farinha de rosca (2 xícaras)\n    *   Molho de tomate (1 sachê ou lata)\n    *   Muçarela fatiada (200g)\n    *   Queijo parmesão ralado (a gosto)\n    *   Óleo (para fritar)\n    *   Tempero a gosto (sal, pimenta, orégano) \n\n**Quarta-feira:**\n\n*   **Café da manhã:**\n    *   Contrafilé (4 bifes)\n    *   Ovos (4 unidades)\n    *   Alho (a gosto)\n    *   Óleo ou manteiga (para fritar)\n    *   Tempero a gosto (sal e pimenta)\n\n**Sexta-feira:**\n\n*   **Jantar:**\n    *   Massa para pastel (500g)\n    *   Queijo muçarela ralado (300g)\n    *   Óleo (para fritar)\n\n**Itens básicos (verifique se precisa repor):**\n\n*   Sal\n*   Pimenta do reino\n*   Azeite\n*   Açúcar \n*   Café \n*   Frutas \n\n**Observações:**\n\n*   As quantidades são apenas sugestões e podem variar de acordo com o apetite e as preferências. \n*   Verifique se possui os utensílios necessários para o preparo dos pratos (ex: panela, frigideira, etc).\n*   Fique à vontade para adaptar o cardápio e a lista de compras conforme suas necessidades.",
  "output: 1 - Cereal: escolha o tipo e a quantidade de acordo com a preferência, ex: 1 caixa de cereal de milho\n2 - Leite: 1 litro\n3 - Macarrão: 500g\n4 - Alho: 1 cabeça\n5 - Óleo: a gosto\n6 - Queijo ralado: a gosto\n7 - Polvilho doce: 500g\n8 - Queijo minas padrão ou meia cura ralado: 200g\n9 - Ovos: 2 unidades\n10 - Óleo: para untar\n11 - Leite: 1/2 xícara\n12 - Filés de frango: 4 unidades\n13 - Ovos: 2 unidades\n14 - Farinha de trigo: 1 xícara\n15 - Farinha de rosca: 2 xícaras\n16 - Molho de tomate: 1 sachê ou lata\n17 - Muçarela fatiada: 200g\n18 - Queijo parmesão ralado: a gosto\n19 - Óleo: para fritar\n20 - Tempero a gosto: sal, pimenta, orégano\n21 - Contrafilé: 4 bifes\n22 - Ovos: 4 unidades\n23 - Alho: a gosto\n24 - Óleo ou manteiga: para fritar\n25 - Tempero a gosto: sal e pimenta\n26 - Massa para pastel: 500g\n27 - Queijo muçarela ralado: 300g\n28 - Óleo: para fritar\n29 - Sal\n30 - Pimenta do reino\n31 - Azeite\n32 - Açúcar \n33 - Café \n34 - Frutas",
  "input: " + convo.last.text,
  "output: ",
]

Executando o modelo, com output somente da lista de compras

In [34]:
response = model_structured.generate_content(prompt_parts)
print(response.text)

1 - Pacote de pão de queijo (500g)
2 - Carne (filé mignon, alcatra ou frango): 500g
3 - Cebola grande: 1 unidade
4 - Alho: 2 dentes
5 - Caixa de creme de leite: 1 unidade
6 - Sachê de molho de tomate: 1 unidade
7 - Vidro de champignon: 1 unidade
8 - Azeite
9 - Sal e pimenta do reino a gosto
10 - Arroz: quantidade de acordo com a preferência
11 - Batata palha: opcional
12 - Bifes grandes (contrafilé ou filé mignon): 4 unidades
13 - Ovos: 2 unidades
14 - Farinha de rosca
15 - Queijo mussarela fatiado
16 - Molho de tomate
17 - Queijo parmesão ralado
18 - Azeite
19 - Sal e pimenta do reino a gosto
20 - Arroz: quantidade de acordo com a preferência
21 - Pacote de pão francês: 5 unidades
22 - Manteiga
23 - Macarrão (espaguete ou penne): 500g
24 - Alho: 4 dentes
25 - Azeite
26 - Sal a gosto
27 - Pimenta do reino a gosto: opcional
28 - Queijo parmesão ralado: opcional 



Possibilidade de carregar imagens de items que voce possui em casa, para eliminar da lista de compras

In [41]:
uploader = FileUpload()
print("Faca upload das imagens dos items que voce tem disponiveis (geladeira, armario, dispensa, etc...)")
display(uploader)

Faca upload das imagens dos items que voce tem disponiveis (geladeira, armario, dispensa, etc...)


FileUpload(value={}, description='Upload')

Metodo auxiliar para carregar os bytes dos arquivos no modelo do Gemini

In [50]:
def save_bytes_as_file(bytes_data, filename=None):
  with tempfile.NamedTemporaryFile(delete=False, suffix=filename) as temp_file:
    temp_file.write(bytes_data)
    return temp_file.name

Carregando os bytes dos arquivos no modelo do Gemini

In [51]:
uploaded_file_data_list = uploader.data
filesUploadedTemp = []
for uploaded_file_data in uploaded_file_data_list:
  filesUploadedTemp.append(genai.upload_file(save_bytes_as_file(uploaded_file_data,"file.jpg")))


Executando o modelo para extrair os items das imagens
  

In [52]:
itemsAtHome = ""
if len(filesUploadedTemp)>0:
  convoImage = model_chat.start_chat(history=[])
  message = filesUploadedTemp+["extraia da imagem uma tabela com nome do item, quantidade e quantidade de conteudo , so retorne a tabela"]
  convoImage.send_message(message)
  itemsAtHome = convoImage.last.text
  print(itemsAtHome)

| Item | Quantidade | Quantidade de Conteúdo |
|---|---|---|
| Trancil | 1 caixa | 20 mg |
| St Dalfour | 1 pote | 284g |
| Vitaminas B | 1 lata |  |
| Heinz Tomato Ketchup | 1 frasco |  |
| Aurora Azeite Extra Virgem | 1 garrafa |  |
| Paçlu | 1 pote |  |
| Leite | 1 caixa |  |
| Shoyu | 1 garrafa |  | 



Executando o modelo para gerar a lista final de compras

In [53]:
convo = model_chat.start_chat(history=[])
finalList = ""
if itemsAtHome == "":
  finalList = response.text
else:
  convo.send_message(["se eu tenho a seguinte lista de compras",response.text,"mas ja tenho os items",itemsAtHome,"qual a minha lista final de compras?", "retorne somente a lista"])
  finalList = convo.last.text
print(finalList)

1. Pacote de pão de queijo (500g)
2. Carne (filé mignon, alcatra ou frango): 500g
3. Cebola grande: 1 unidade
4. Alho: 6 dentes 
5. Caixa de creme de leite: 1 unidade
6. Sachê de molho de tomate: 2 unidades 
7. Vidro de champignon: 1 unidade
8. Sal e pimenta do reino a gosto
9. Arroz: quantidade de acordo com a preferência
10. Batata palha: opcional
11. Bifes grandes (contrafilé ou filé mignon): 4 unidades
12. Ovos: 2 unidades
13. Farinha de rosca
14. Queijo mussarela fatiado
15. Queijo parmesão ralado
16. Pacote de pão francês: 5 unidades
17. Manteiga
18. Macarrão (espaguete ou penne): 500g 

